In [1]:
from datetime import datetime
from git import Repo, Git
import subprocess
import os
import csv

filename = "C:\\Users\\leu_m\\PycharmProjects\\master-terraform-sec-adoption\\google\\csv\\google-analysis-tfsec.csv"

METRICS_FILE = "C:\\Users\\leu_m\\PycharmProjects\\master-terraform-sec-adoption\\google\\csv\\google_dataset_metrics.csv"
STAR_FILTERED_REPOS = "C:\\Users\\leu_m\\PycharmProjects\\master-terraform-sec-adoption\\google\\csv\\google_dataset_filtered.csv"
INPUT_FILE = "C:\\Users\\leu_m\\PycharmProjects\\master-terraform-sec-adoption\\google\\csv\\google_results-tfsec.csv"
PATH_OF_TMP_DIRECTORY = "C:\\Users\\leu_m\\PycharmProjects\\master-terraform-sec-adoption\\tmp\\google"

stars_by_projects = {}
forks_by_projects = {}
contrib_by_projects = {}

with open(STAR_FILTERED_REPOS, 'r') as f:
    tmp_repo_names = f.readlines()
repo_names = [k.strip("\n") for k in tmp_repo_names]

with open(METRICS_FILE, 'r') as f:
    #f.readline()
    rows = f.readlines()
    for item in rows:
        column = item.split(',')
        if column[0] in repo_names:
            stars_by_projects[column[0]] = int(column[1])
            forks_by_projects[column[0]] = int(column[2])
            contrib_by_projects[column[0]] = int(column[3].strip("\n"))

with open(INPUT_FILE, 'r') as f:
    f.readline()
    repo_results = f.readlines()

repo_stars_mine = {}
with open(METRICS_FILE, 'r') as f:
    f.readline()
    rows = f.readlines()
    for item in rows:
        column = item.split(',')
        if column[0] in repo_names:
            repo_stars_mine[column[0]] = column[1]

stars_passed_rate_mine = {}
for row in repo_results:
    columns = row.split(',')
    repo_name = columns[0]
    if repo_name in repo_stars_mine:
        passed_count = len(columns[3].split(';'))
        failed_count = len(columns[4].split(';'))
        if passed_count + failed_count > 0:
            stars_passed_rate_mine[repo_name] = passed_count/(passed_count+failed_count) 

def get_commit_count_until_hash(repo_name):
    try:
        # Create the directory if it doesn't exist
        if not os.path.exists(PATH_OF_TMP_DIRECTORY):
            os.makedirs(PATH_OF_TMP_DIRECTORY)
        
        # Change directory to the specified directory
        os.chdir(PATH_OF_TMP_DIRECTORY)
        
        # Clone the repository
        repo_url = "https://github.com/"+repo_name+".git"
        subprocess.run(["git", "clone", repo_url])
        repo_name = str(repo_name.split("/")[1]).replace(".git","")
        
        # Change directory to the cloned repository
        os.chdir(os.path.join(PATH_OF_TMP_DIRECTORY, repo_name))
        
        commit_hash = get_closest_commit(os.path.join(PATH_OF_TMP_DIRECTORY, repo_name), datetime(2023, 4, 26))
        
        # Fetch all commits and count them
        output = subprocess.check_output(["git", "rev-list", "--count", commit_hash]).decode("utf-8").strip()
        commit_count = int(output)
        
        output = subprocess.check_output(["git", "log", "--format=%aN", commit_hash]).decode("utf-8")
        contributors = len(set(output.splitlines()))
        
        return commit_count, contributors
    
    except subprocess.CalledProcessError as e:
        print("Error:", e)
        return None

def get_closest_commit(repo_path, target_date):
    git = Git(repo_path)

    # Use rev-list to find the closest commit to the target date
    closest_commit_hash = git.rev_list('-n', '1', '--before={:%Y-%m-%d %H:%M:%S}'.format(target_date), '--all')

    return closest_commit_hash

with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["project_name", "stars_passed", "stars", "forks", "contributors", "commits", "commiters"])
        for row in stars_passed_rate_mine.keys():
            try:
                report = get_commit_count_until_hash(row)
                commit_count = report[0]
                contributors = report[1]
                
                if commit_count is None:
                    commit_count = 0
                    
                if contributors is None:
                    contributors = 0
                    
                csv_writer.writerow([row, stars_passed_rate_mine.get(row), stars_by_projects.get(row), 
                                    forks_by_projects.get(row), contrib_by_projects.get(row), commit_count, contributors])
            except Exception as e:
                print(row)

In [2]:
import csv
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd

filename = "C:\\Users\\leu_m\\PycharmProjects\\master-terraform-sec-adoption\\google\\csv\\google-analysis-tfsec.csv"

# Sample DataFrame
data = pd.read_csv(filename)

# Create DataFrame
df = pd.DataFrame(data)

# Function to map passed rate to categories
def map_passed_rate(passed_rate):
    if passed_rate < 0.3:
        return "LOW_RATE"
    #elif passed_rate < 0.4:
    #    return "LOW_RATE"
    elif passed_rate < 0.6:
        return "MEDIUM_RATE"
    #elif passed_rate < 0.8:
     #   return "HIGH_RATE"
    else:
        return "HIGH_RATE"

    # Function to map other continuous variables to categories based on min-max normalization
def map_to_categories(value, min_val, max_val, category):
    if value < min_val + 0.1 * (max_val - min_val):
        return "VERY_LOW_"+category
    elif value < min_val + 0.2 * (max_val - min_val):
        return "LOW_"+category
    elif value < min_val + 0.3 * (max_val - min_val):
        return "MODERATE_"+category
    elif value < min_val + 0.4 * (max_val - min_val):
        return "BELOW_AVERAGE_"+category
    elif value < min_val + 0.5 * (max_val - min_val):
        return "AVERAGE_"+category
    elif value < min_val + 0.6 * (max_val - min_val):
        return "ABOVE_AVERAGE_"+category
    elif value < min_val + 0.7 * (max_val - min_val):
        return "HIGH_"+category
    elif value < min_val + 0.8 * (max_val - min_val):
        return "VERY_HIGH_"+category
    elif value < min_val + 0.9 * (max_val - min_val):
        return "EXTREMELY_HIGH_"+category
    else:
        return "EXCEPTIONALLY_HIGH_"+category

# Transforming passed rate
df['passed_rate_category'] = df['stars_passed'].apply(map_passed_rate)

# Transforming stars, forks, and contributors
for col in ['stars', 'forks', 'commits', 'commiters']:
    min_val = df[col].min()
    max_val = df[col].max()
    df[col + '_category'] = df[col].apply(lambda x: map_to_categories(x, min_val, max_val, col))

# Print the transformed DataFrame
#print(df.columns)

print(df[['passed_rate_category', 'stars_category', 'forks_category', 'commits_category', 'commiters_category']])

aux = df[['passed_rate_category', 'stars_category', 'forks_category', 'commits_category', 'commiters_category']]

transactions = aux.values

print(transactions)

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_selected = pd.DataFrame(te_ary, columns=te.columns_)

# Generate frequent itemsets with minimum support of 0.3
frequent_itemsets = apriori(df_selected, min_support=0.3, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Print the rules
for index, rule in rules.iterrows():
    antecedents = ', '.join(rule['antecedents'])
    consequents = ', '.join(rule['consequents'])
    print(f"Rule: {antecedents} -> {consequents}")
    print(f"Support: {rule['support']}")
    print(f"Confidence: {rule['confidence']}")
    print(f"Lift: {rule['lift']}")
    print()
    

filename = "C:\\Users\\leu_m\\PycharmProjects\\master-terraform-sec-adoption\\rq3\\report-google-tfsec.csv"

with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["antecedents", "consequents", "Support", "Confidence", "Lift"])
        for index, rule in rules.iterrows():
            antecedents = ', '.join(rule['antecedents'])
            consequents = ', '.join(rule['consequents'])
            csv_writer.writerow([antecedents, consequents, rule['support'], rule['confidence'], rule['lift']])
            

    passed_rate_category            stars_category       forks_category  \
0            MEDIUM_RATE  EXCEPTIONALLY_HIGH_stars       VERY_LOW_forks   
1            MEDIUM_RATE      EXTREMELY_HIGH_stars  BELOW_AVERAGE_forks   
2            MEDIUM_RATE      EXTREMELY_HIGH_stars       VERY_LOW_forks   
3               LOW_RATE           VERY_HIGH_stars       VERY_LOW_forks   
4              HIGH_RATE                HIGH_stars       VERY_LOW_forks   
..                   ...                       ...                  ...   
237            HIGH_RATE            VERY_LOW_stars       VERY_LOW_forks   
238            HIGH_RATE            VERY_LOW_stars       VERY_LOW_forks   
239          MEDIUM_RATE            VERY_LOW_stars       VERY_LOW_forks   
240          MEDIUM_RATE            VERY_LOW_stars       VERY_LOW_forks   
241            HIGH_RATE            VERY_LOW_stars       VERY_LOW_forks   

     commits_category       commiters_category  
0    VERY_LOW_commits       VERY_LOW_commiters  
1